## Parte 1: Predicción del valor de mercado

*a) Recta de regresión para predecir el valor de mercado de un jugador a partir de la característica más relevante (a la que se destinará mayor proporción del presupuesto), respaldada por:*

Consideramos como caracteristica mas relevante al overall, que es la puntuacion general del jugador y por lo tanto, que tan buena figurita es.
A partir de esta muestra de datos graficamos la relacion entre el valor de mercado(y) y el overall(x).
No sin antes normalizar los valores de mercado, es decir, establecerlos en un rango que sea mas legible y entendible a la hora de graficar o ver estadisticas.
Tambien consideramos no poner aquellos valores con overall muy bajo ya que representan una escasa cantidad de figuritas que no aportan relevancia al analisis.

In [ ]:
from numpy import sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# Cargar el dataset
df = pd.read_csv("code/dataset/FIFA21.csv")

relevant_feature = "overall"
# Seleccionar las columnas de interés
df = df[["long_name", "value_eur", relevant_feature]].dropna()

# Eliminar todos los valores cuyo overall sea menor a 65
df = df[df[relevant_feature] > 65]

# Normalizar la columna 'value_eur' a un rango de 0 a 100
df['value_eur_normalized'] = 100 * (df['value_eur'] - df['value_eur'].min()) / (df['value_eur'].max() - df['value_eur'].min())

# Definir las variables
x = df[relevant_feature]
y = df['value_eur_normalized']

# Grafico con los datos obtenidos
plt.scatter(x, y, color='red')
plt.title('Value vs Overall')
plt.xlabel('Overall')
plt.ylabel('Value')
plt.show()

De esta forma ya podemos ver que tiene cierta relacion entre el overall y el valor de mercado ya que al aumentar uno se puede ver el aumento del otro asi que podemos avanzar con la recta estimativa de valores.

Para esto sacamos todos los valores necesarios para obtener una ecuacion de la recta tal que 
### Y = β1X + β0 + ε

In [ ]:
# Recta de regresión para predecir el valor de mercado
# de un jugador a partir de la característica más relevante

# Calcular la sumatoria de las variables x e y
x_sum = x.sum()
y_sum = y.sum()

# Calcular la media de las columnas 2 y 3
x_avg = x.mean()
y_avg = y.mean()

# Calcular la sumatoria de los cuadrados de las variables x e y
x_sq_sum = (x ** 2).sum()
y_sq_sum = (y ** 2).sum()

# Sigma al cuadrado de x e y (varianza corregida)
x_var = x_sq_sum / len(x) - (x_sum ** 2) / len(x)**2
y_var = y_sq_sum / len(y) - (y_sum ** 2) / len(y)**2

# Sigma de xy
xy_dev = ((x * y).sum() / len(x)) - (x_avg * y_avg)

# Coeficientes de la recta de regresión Pendiente y Ordenada
b1 = xy_dev / x_var
b0 = y_avg - b1 * x_avg

# Estimación de mínimos cuadrados
Sxy = (x * y).sum() - ((x.sum() * y.sum()) / len(x))
Sxx = x_sq_sum - ((x.sum() ** 2) / len(x))
Syy = y_sq_sum - ((y.sum() ** 2) / len(y))

# Coeficientes de la recta de regresión Pendiente y Ordenada
b1_hat = Sxy / Sxx
b0_hat = y_avg - b1_hat * x_avg

y_hat = b0_hat + b1_hat * x

# Dibujar recta a partir de y_hat
plt.scatter(x, y, color='red', label='Datos')
plt.plot(x, y_hat, color='blue', label='Recta de regresión')
plt.title('Value vs ' + relevant_feature + 'con Recta de Regresión')
plt.xlabel(relevant_feature)
plt.ylabel('Value Normalized')
plt.legend()
plt.show()

*i) Prueba de significancia de regresión, coeficiente de determinación (R²) y correlación lineal
(r)*

In [ ]:
SSr = Syy - (b1_hat * Sxy)

# Calculo de la varianza
sigma_hat_sqrd = SSr / (len(x) - 2)

# Calculo del coeficiente de determinación
R2 = 1 - SSr/Syy

# Coeficiente de Correlación Lineal
r = Sxy / sqrt(Sxx * Syy)

# Imprimir los resultados
print("SSr: " + str(SSr))
print("Varianza: " + str(sigma_hat_sqrd))
print("Coeficiente de determinacion: " + str(R2 * 100) + "%")
print("Coeficiente de correlación: " + str(r) + "\n")

# Prueba de significancia
# Se calcula el error estandar de la pendiente
dflen = len(x) - 2

sse = ((y - y_hat) ** 2).sum()
error_estandar_pendiente = sqrt(sse / (dflen * Sxx))

# Test de Hipótesis
t = b1_hat / error_estandar_pendiente

print("Error estandar de la pendiente: " + str(error_estandar_pendiente))
print("Test de Hipótesis: " + str(t))

Sobre el coeficiente de determinacion este porcentaje indica que el modelo explica el 63.65% de la variabilidad total en los datos. Si bien no es perfecto (100%), sigue siendo un buen indicador de que el modelo tiene una capacidad razonable para predecir el comportamiento de los datos.

Por otra parte el coeficiente de correlacion refleja la fuerza y la dirección de la relación entre las variables independientes y dependientes. Un valor de 0.7978 indica una fuerte correlación positiva, lo que significa que a medida que una variable aumenta, la otra también tiende a aumentar.

Error estandar de la pendiente: 0.008869301837671184 , este valor indica la precisión de la pendiente estimada. Un error estándar bajo sugiere que la pendiente está bien estimada y es confiable.

Test de hipotesis; 129.40 Este valor es un estadístico t que evalúa si la pendiente es significativamente diferente de 0. Un valor tan alto indica que la pendiente es significativamente distinta de 0, lo que respalda la existencia de una relación significativa entre las variables.

*ii) Inferencias sobre los parámetros de la recta, estimando las
fluctuaciones con una confianza del 95%.
Conclusión del test*


In [ ]:

# Valor de P
p = 2 * (1 - stats.t.cdf(abs(t), dflen))

if p < 0.05:
    print("Se rechaza la hipótesis nula")
else:
    print("No se rechaza la hipótesis nula")

Esto significa que la pendiente es significativamente diferente de 0, y por lo tanto, existe una relación significativa entre las variables. Si el valor p fuese mayor o igual a 0.05, no se puede rechazar la hipótesis nula, lo que implica que la pendiente podría no ser significativa.

*iii) La proporción de veces que el valor de mercado supera la incertidumbre de predicción
comparada con la respuesta media del valor de mercado para una característica fija, ambas
con la misma confianza y ancho mínimo.*

In [10]:
alpha = 0.05
t_alpha_half = stats.t.ppf(1 - alpha / 2, dflen)  # Valor crítico de t
x0 = x.mean()  # Característica fija, puedes cambiar a otro valor específico

# Predicción para x0
y0_hat = b0_hat + b1_hat * x0

# Incertidumbre de predicción
pred_interval = t_alpha_half * \
    sqrt(sigma_hat_sqrd * (1 + 1 / len(x) + (x0 - x_avg)**2 / Sxx))

# Intervalo de confianza para la media
conf_interval = t_alpha_half * \
    sqrt(sigma_hat_sqrd / len(x) + (x0 - x_avg)**2 / Sxx)

# Calcular la proporción de veces que el valor de mercado supera la
# incertidumbre de predicción
exceeds_prediction_interval = np.sum(
    (y > (y0_hat + pred_interval)) | (y < (y0_hat - pred_interval)))
proportion_exceeds = exceeds_prediction_interval / len(y)

# Imprimir resultados
print(f"Intervalo de Predicción: ±{pred_interval}")
print(f"Intervalo de Confianza para la Media: ±{conf_interval}")
print(
    f"Proporción de veces que el valor de mercado supera la incertidumbre de predicción: {proportion_exceeds:.2f}")

Intervalo de Predicción: ±7.486234728661791
Intervalo de Confianza para la Media: ±0.0765417261143835
Proporción de veces que el valor de mercado supera la incertidumbre de predicción: 0.07


La incertidumbre de predicción (±7.49) es considerablemente mayor que la incertidumbre en la media (±0.08), lo que es de esperarse, ya que predecir un nuevo valor individual es más incierto que estimar la media de muchos valores. La baja proporción (0.07) indica que en la mayoría de los casos, el valor de mercado real está dentro del rango de predicción, lo que sugiere que el modelo tiene un buen desempeño al predecir el mercado, aunque puede haber algunas fluctuaciones.